# AdaBoost y Gradient boosting con árboles en un dataset de detección de spam

Árboles de decisión y todos los ensembles propuestos funcionan en la práctica muy bien en datos tabulares como puede ser el problema de reconocimiento de spam en mails.

Se propone pues emplear el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from warnings import filterwarnings

## Descarga del dataset Spam
filterwarnings('ignore')
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)



(4601, 57)


## AdaBoost

Probemos el clasificador de AdaBoost

In [3]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier()

acc=ab.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Precisión: 93.8%


**Ejercicio** Teniendo en cuenta la documentación [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) prueba mediante grid search a encontrar mejores parámetros.

In [4]:
from sklearn.model_selection import GridSearchCV

G = {"n_estimators": [2, 5, 10, 20, 50], "learning_rate": [0.01, 0.1, 0.3, 0.5, 0.7, 1]}

GS = GridSearchCV(ab, G, scoring='accuracy', refit=True, cv=5)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 94.9% con {'learning_rate': 0.5, 'n_estimators': 50}


## GradientBoostingClassifier

Probemos el clasificador Gradient Boosting

In [4]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()

acc=gb.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')


Precisión: 95.0%


Los principales parámetros del GradientBoosting son:
* n_estimators (100)
* max_depth (3)
* max_features

**Ejercicio** prueba diferentes valores (lógicos) de estos parámetros mediante GridSearch.

In [7]:
from sklearn.model_selection import GridSearchCV

G = {"n_estimators": [2, 5, 10, 20, 50], "max_depth": [2, 3, 5, 10, 20], "max_features":[10, 20, 30, 40, 50]}

GS = GridSearchCV(gb, G, scoring='accuracy', refit=True, cv=5)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 95.7% con {'max_depth': 10, 'max_features': 20, 'n_estimators': 50}


### Acelerar la búsqueda de parámetros: RandomizedSearch

Un problema recurrente que tenemos en estos experimentos es el coste temporal de encontrar los mejores parámetros mediante GridSearch. Una alternativa que reduce dicho coste temporal es el [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

**Ejercicio** Implementa la misma búsqueda de valores que has realizado con GridSearch pero ahora con RandomizedSearch con n_iter=10 para limitar el número de pruebas a 10.

In [5]:
from sklearn.model_selection import RandomizedSearchCV

G = {"max_depth": [1,2,3,4,5],"n_estimators":[50,100,150,200],"max_features":[10,20,30,40,57]}

# Probar sólo 10 combinaciones de parámetros, n_iter=10
RS = RandomizedSearchCV(GradientBoostingClassifier(), G, scoring='accuracy', n_iter=10, refit=True, cv=5, verbose=10)

acc = RS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {RS.best_params_}')


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START max_depth=1, max_features=57, n_estimators=150.............
[CV 1/5; 1/10] END max_depth=1, max_features=57, n_estimators=150;, score=0.942 total time=   1.8s
[CV 2/5; 1/10] START max_depth=1, max_features=57, n_estimators=150.............
[CV 2/5; 1/10] END max_depth=1, max_features=57, n_estimators=150;, score=0.924 total time=   1.0s
[CV 3/5; 1/10] START max_depth=1, max_features=57, n_estimators=150.............
[CV 3/5; 1/10] END max_depth=1, max_features=57, n_estimators=150;, score=0.944 total time=   1.0s
[CV 4/5; 1/10] START max_depth=1, max_features=57, n_estimators=150.............
[CV 4/5; 1/10] END max_depth=1, max_features=57, n_estimators=150;, score=0.933 total time=   1.1s
[CV 5/5; 1/10] START max_depth=1, max_features=57, n_estimators=150.............
[CV 5/5; 1/10] END max_depth=1, max_features=57, n_estimators=150;, score=0.938 total time=   1.5s
[CV 1/5; 2/10] START max_depth=3, max_f

### Mejorar la búsqueda de parámetros: BayesianOptimization

Optimización Bayesiana es una técnica avanzada de búsqueda de parámetros. Para poder emplearla hay que instalar la librería scikit-optimize. El uso de dicha técnica es similar a las ya empleadas con la diferencia que el rango de los parámetros hay que definirlos teniendo en cuenta además el tipo de los mismos: real, entero o categórico. Además los valores a probar no se listan explícitamente si no que se especifica un rango dentro del cual el algoritmo escogerá los valores.

La ejecución puede resultar más lenta pero se pueden explorar más parámetros, de hecho se especifica un intervalo en lugar de valores concretos.



In [8]:
!pip install scikit-optimize

  Obtaining dependency information for scikit-optimize from https://files.pythonhosted.org/packages/90/0e/15deb91b3db0003843e34e72fa865e1d92013781d986fdc65483c99a9f69/scikit_optimize-0.10.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyaml>=16.9 from https://files.pythonhosted.org/packages/70/d9/6fdc01cee01fe56fddfd46892d9846ebb53a5f8eb78056955c516eef42c5/pyaml-24.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/107.7 kB ? eta -:--:--
   --- ------------------------------------ 10.2/107.7 kB ? eta -:--:--
   ---------------------------------------- 107.7/107.7 kB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Ponemos rangos (mínimo, máximo)
G = {"max_depth": Integer(1,5),"n_estimators":Integer(50,400),"max_features":Integer(5,57)}

# Probar solo 10 combinaciones de parámetros, n_iter=10
BS = BayesSearchCV(GradientBoostingClassifier(), G, scoring='accuracy', n_iter=10, refit=True, cv=5, verbose=10)
acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START max_depth=1, max_features=24, n_estimators=86...............
[CV 1/5; 1/1] END max_depth=1, max_features=24, n_estimators=86;, score=0.931 total time=   0.4s
[CV 2/5; 1/1] START max_depth=1, max_features=24, n_estimators=86...............
[CV 2/5; 1/1] END max_depth=1, max_features=24, n_estimators=86;, score=0.910 total time=   0.6s
[CV 3/5; 1/1] START max_depth=1, max_features=24, n_estimators=86...............
[CV 3/5; 1/1] END max_depth=1, max_features=24, n_estimators=86;, score=0.933 total time=   0.4s
[CV 4/5; 1/1] START max_depth=1, max_features=24, n_estimators=86...............
[CV 4/5; 1/1] END max_depth=1, max_features=24, n_estimators=86;, score=0.928 total time=   0.3s
[CV 5/5; 1/1] START max_depth=1, max_features=24, n_estimators=86...............
[CV 5/5; 1/1] END max_depth=1, max_features=24, n_estimators=86;, score=0.921 total time=   0.4s
Fitting 5 folds for each of 1 candidates, totalling

## HistGradientBoostingClassifier
Un problema del clasificador GradientBoosting implementado en sklearn es su velocidad. Es un clasificador muy lento de entrenar. Por ello sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma.

Este otro algoritmo se denomina **HistGradientBoostingClassifier**. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



In [12]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier()

acc=hgb.fit(X_train, y_train).score(X_test, y_test)

print(f'Precisión: {acc:.1%}')

Precisión: 95.5%


**Ejercicio** Analiza en la documentación [HistGradientBoosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html) los parámetros más relevantes y realiza una búsqueda de parámetros para obtener el mejor clasificador. A la hora de establecer los valores de los parámetros sería interesante fijarnos en los valores por defecto que se han empleado en el ejercicio anterior y poder explorar alrededor de dichos valores por defecto para conseguir mejores resultados.



In [13]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

G = {"max_iter":Integer(50,400),"max_bins":Integer(10,255)}

# Probar solo 10 combinaciones de parámetros, n_iter=10
BS = BayesSearchCV(hgb, G, scoring='accuracy', n_iter=10, refit=True, cv=5, verbose=10)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START max_bins=45, max_iter=230...................................
[CV 1/5; 1/1] END ....max_bins=45, max_iter=230;, score=0.946 total time=   3.7s
[CV 2/5; 1/1] START max_bins=45, max_iter=230...................................
[CV 2/5; 1/1] END ....max_bins=45, max_iter=230;, score=0.963 total time=   3.7s
[CV 3/5; 1/1] START max_bins=45, max_iter=230...................................
[CV 3/5; 1/1] END ....max_bins=45, max_iter=230;, score=0.969 total time=   3.7s
[CV 4/5; 1/1] START max_bins=45, max_iter=230...................................
[CV 4/5; 1/1] END ....max_bins=45, max_iter=230;, score=0.951 total time=   3.8s
[CV 5/5; 1/1] START max_bins=45, max_iter=230...................................
[CV 5/5; 1/1] END ....max_bins=45, max_iter=230;, score=0.957 total time=   4.0s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START max_bins=19, max_iter=220...........................

In [14]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

G = {"max_iter":Integer(50,400),"max_bins":Integer(10,255), "max_depth": Integer(2,10), "learning_rate": Real(0,1)}

# Probar solo 10 combinaciones de parámetros, n_iter=10
BS = BayesSearchCV(hgb, G, scoring='accuracy', n_iter=10, refit=True, cv=5, verbose=10)

acc = BS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {BS.best_params_}')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121
[CV 1/5; 1/1] END learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121;, score=0.946 total time=   0.1s
[CV 2/5; 1/1] START learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121
[CV 2/5; 1/1] END learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121;, score=0.950 total time=   0.1s
[CV 3/5; 1/1] START learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121
[CV 3/5; 1/1] END learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121;, score=0.961 total time=   0.1s
[CV 4/5; 1/1] START learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121
[CV 4/5; 1/1] END learning_rate=0.1666036872051291, max_bins=23, max_depth=3, max_iter=121;, score=0.946 total time=   0.1s
[CV 5/5; 1/1] START learning_rate=0.1666036872051291, max_bins=23, max_de